# Image classification task

- Alexander Pasha
-Edward Long

In [ ]:
! pip install tensorflow

## Import data from kaggle
(necessary if using colab, place kaggle json in the content folder)

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
# obtain a kaggle API token from your account and place it in the content folder
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download prashant268/chest-xray-covid19-pneumonia/code

In [ ]:
!unzip chest-xray-covid19-pneumonia.zip

# Transfer learning models
## VGG16 and ResNet50 models

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, RandomContrast, RandomRotation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import vgg16, resnet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import os

In [ ]:
data_path = 'Data/'
num_classes = 3
classes = ['Covid','Normal', 'Pneumonia']

In [ ]:
def build_transfer_model(Model, num_classes, opt=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy') -> Sequential:
    """builds a model from one of the Model types in tensorflow.python.keras.applications"""
    model = Sequential()
    model.add(Model(include_top=False, weights='imagenet', pooling='avg', input_shape=(128,128,3)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.layers[0].trainable = False
    model.compile(optimizer=opt, loss=loss, metrics=['sparse_categorical_accuracy'])
    return model

In [ ]:
def build_generator(data_path, preprocessing_function,img_size=128,batch_size=128):
    """Builds a generator 
    if for the purpose of training this returns a tuple: training and validation sets,
    if for the purpose of testing this returns a single generator: testing set
    """
    train_path = os.path.join(data_path, 'train')
    test_path = os.path.join(data_path, 'test')

    # test 
    train_generator_instance = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        validation_split=0.2)
    train_generator = train_generator_instance.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        class_mode='sparse',
        subset='training',
        batch_size=batch_size
    )
    val_generator = train_generator_instance.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        class_mode='sparse',
        subset='validation',
        batch_size=batch_size
    )
    # Test generator
    test_generator_instance = ImageDataGenerator(
        preprocessing_function=preprocessing_function)
    test_generator = test_generator_instance.flow_from_directory(
        test_path,
        target_size=(img_size, img_size),
        class_mode='sparse',
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_generator, val_generator,test_generator

In [ ]:
def train_model(model, preprocess_function, data_path, model_name='', epochs = 10):
    """train a model using an instance of a model, its preprocessor, the path to the data and a name for outputs"""


    X_train,X_val,X_test = build_generator(data_path, preprocess_function)


    y_test = X_test.classes

    # history = model.fit(X_train, validation_data=X_val, verbose=1, steps_per_epoch=X_train.samples//no_of_epochs, epochs = no_of_epochs)
    history = model.fit(X_train, validation_data=X_val, verbose=1, epochs = epochs)
    y_pred = np.argmax(model.predict(X_test, verbose=1),axis=-1)
    print(y_pred.shape)
    print('Classification Report of ', model_name)
    report = classification_report(y_test, y_pred, target_names=classes)
    # need to add labels, but had some problems here before
    print(report)
    return history

In [ ]:
# train and report resnet model
resnet =build_transfer_model(resnet50.ResNet50,num_classes)
resnet_history = train_model(resnet, resnet50.preprocess_input, data_path, model_name='ResNet50')

In [ ]:
# train and report vgg16 model
vgg = build_transfer_model(vgg16.VGG16, num_classes)
vgg_history = train_model(vgg, vgg16.preprocess_input, data_path, model_name = 'VGG16')

# New model
Resnet+

In [ ]:
model = Sequential()
model.add(RandomContrast(0.2))
model.add(RandomRotation(0.2, fill_mode="constant"))
model.add(resnet50.ResNet50(include_top=False, input_shape=(128,128,3), weights='imagenet'))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))

model.add(Dense(3, activation='softmax'))
model.layers[2].trainable = True
model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy',metrics = ['sparse_categorical_accuracy'])

In [ ]:
# model.load_weights("resNet.h5")
X_train, X_val, X_test = build_generator(data_path, resnet50.preprocess_input)

In [ ]:
input_shape = (None, 128,128,3)
model.build(input_shape)
model.summary()

In [ ]:
# model.load_weights('/content/new_model_wieghts.h5')

In [ ]:
history = model.fit(
    X_train,
    validation_data=X_val,
    epochs=100,
    verbose=1,
    callbacks=EarlyStopping(monitor='val_loss', mode='min', patience=25, restore_best_weights=True)
)

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy']].plot();

In [ ]:
y_test = X_test.classes
y_pred = np.argmax(model.predict(X_test, verbose=1),axis=-1)

print('Classification Report of ', 'ResNet+')
report = classification_report(y_test, y_pred, target_names=classes)
# need to add labels, but had some problems here before
print(report)

In [ ]:
test_scores = model.evaluate(X_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
# model.save_weights("/content/new_model_wieghts.h5")

In [ ]:
# from google.colab import files
# files.download("/content/new_model_wieghts.h5")